In [1]:
%load_ext autoreload
%env SPARK_HOME=/usr/hdp/current/spark2-client

import findspark
findspark.init()

print('findspark initialized ...')

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr, col, column, max, min

spark = SparkSession.builder.appName('mlonspark')\
    .config('spark.executor.instances', '8')\
    .config('spark.jars', '/opt/dev/target/ml-on-spark-1.0.jar')\
    .getOrCreate()

#   
    
print('pyspark ready ...')

env: SPARK_HOME=/usr/hdp/current/spark2-client
findspark initialized ...
pyspark ready ...


In [2]:
train = spark.read.format("parquet").load("/data/books/ratings-train.parquet")
train.printSchema()

root
 |-- User-ID: string (nullable = true)
 |-- ISBN: string (nullable = true)
 |-- Book-Rating: string (nullable = true)
 |-- rating: float (nullable = true)
 |-- bookId: integer (nullable = true)
 |-- Location: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- userId: integer (nullable = true)



In [3]:
%autoreload
from mlonspark.alternating_least_square import AlternatingLeastSquare
#from mlonspark.alternating_least_square import XALS

als = AlternatingLeastSquare()\
    .setUserCol("userId")\
    .setItemCol("bookId")\
    .setRatingCol("rating")

print(als)

AlternatingLeastSquare_451184b0e43d4f21db00


In [4]:
model = als.fit(train)
print(model)

AlternatingLeastSquare_451184b0e43d4f21db00


In [5]:
model.write().overwrite().save('/data/books/model.alsmodel')

In [6]:
%autoreload
#from mlonspark.alternating_least_square import AlternatingLeastSquareModel

#model = AlternatingLeastSquareModel.read().load('/data/books/model.alsmodel')

In [7]:
print(model)

AlternatingLeastSquare_451184b0e43d4f21db00


In [8]:
test = spark.read.format("parquet").load("/data/books/ratings-test.parquet")
test.printSchema()

root
 |-- User-ID: string (nullable = true)
 |-- ISBN: string (nullable = true)
 |-- Book-Rating: string (nullable = true)
 |-- rating: float (nullable = true)
 |-- bookId: integer (nullable = true)
 |-- Location: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- userId: integer (nullable = true)



In [9]:
predictions = model.transform(test)
predictions.printSchema()
predictions.show(10)

root
 |-- User-ID: string (nullable = true)
 |-- ISBN: string (nullable = true)
 |-- Book-Rating: string (nullable = true)
 |-- rating: float (nullable = true)
 |-- bookId: integer (nullable = true)
 |-- Location: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- userId: integer (nullable = true)
 |-- prediction: float (nullable = false)

+-------+----------+-----------+------+------+--------------------+----+------+-------------+
|User-ID|      ISBN|Book-Rating|rating|bookId|            Location| Age|userId|   prediction|
+-------+----------+-----------+------+------+--------------------+----+------+-------------+
|  27813|0971880107|          0|   1.0|    26|  boerne, texas, usa|  34| 27813|  0.026998185|
| 150364|0971880107|          0|   1.0|    26|santo domingo, n/...|  36|150364|  0.026749456|
| 243930|0971880107|          0|   1.0|    26|montreuil, seine ...|  43|243930|   0.30657512|
|  43058|0971880107|          0|   1.0|    26|bridport, england...|NULL| 43058|2

In [10]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.functions import isnan

evaluator = RegressionEvaluator()\
    .setMetricName("rmse")\
    .setLabelCol("rating")\
    .setPredictionCol("prediction")

predictionsFiltered = predictions.where(~isnan(col("prediction")))
                                
print("number of predictions = %i" % predictionsFiltered.count())

rmse = evaluator.evaluate(predictionsFiltered)

print("RMSE = %f" % rmse)

number of predictions = 279936
RMSE = 1.622174


In [14]:
from pyspark.ml import Pipeline
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.recommendation import ALS

#als2 = XALS()\
#    .setUserCol("userId")\
#    .setItemCol("bookId")\
#    .setRatingCol("rating")

pipeline = Pipeline().setStages([als])

params = ParamGridBuilder()\
    .addGrid(als.rank, range(6, 24, 2))\
    .build()
#    .addGrid(als.rank, [2, 5, 10])\
    

cv = CrossValidator()\
    .setEstimator(als)\
    .setEstimatorParamMaps(params)\
    .setEvaluator(evaluator)\
    .setNumFolds(5)

model = cv.fit(train)

In [15]:
model.bestModel
model.avgMetrics

[1.6209561629874365,
 1.6205474326224263,
 1.6197064651447595,
 1.6194483412184253,
 1.619152001040322,
 1.6187775703541334,
 1.6186348739480114,
 1.618588836144526,
 1.6183966092557387]

In [16]:
bestModel = model.bestModel

bestModel.write().overwrite().save('/data/books/bestModel-rank.model')

In [17]:
predictionsBestModel = bestModel.transform(test)
bestModelRmse = evaluator.evaluate(predictionsBestModel)

print("RMSE = %f" % bestModelRmse)

RMSE = 1.619105
